Keras (ja TensorFlow) perusteet
======================
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/TensorFlowLogo.svg/1229px-TensorFlowLogo.svg.png" 
alt="TensorFlow" width="400"/>
![Keras](https://upload.wikimedia.org/wikipedia/commons/c/c9/Keras_Logo.jpg "Keras")

Asentaaksesi tarvittavat paketit omalla koneellasi harjoituksen suorittamista varten:
```
$ pip3 install scikit-learn pandas tensorflow numpy matplotlib==2.2.2
```

## Keras
Nykyisellään TensorFlow suosittelee aloittelijoita lähtemään liikkeelle juuri [Keras](https://keras.io/)-API:n avulla. Keras sisältää kaksi hieman erityyppistä lähestymistapaa neuroverkkoihin: 
* [Sequential](https://keras.io/getting-started/sequential-model-guide/) on yksinkertaisempi, lineaarinen kasa neurotasoja. Tässä harjoituksessa keskitytään Sequential-API:in.
* [Functional](https://keras.io/getting-started/functional-api-guide/) mahdollistaa monmimutkaisempien mallien rakentamisen, muun muassa mahdollistamalla useampia syöte- ja tulostetasoja

Koska tässä harjoituksessa käytetään Kerasta TensoFlow:n kautta, Keras-dokumentaatiosta poiketen moduulien import-lauseet ovat hieman erilaiset. Esimerkiksi sen sijaan, että kirjoitettaisiin 
``` python
from keras.layers import Dense
layer = Dense(32, input_shape=(784,))
```
kirjoitetaankin
``` python
from tensorflow import keras
layer = keras.layers.Dense(32, input_shape=(784,))
```

## Yksinkertainen neuroverkko
Aluksi tehdään yksinkertainen luokittelu käyttäen hyväksi jo tutuksi tullutta [MNIST](https://en.wikipedia.org/wiki/MNIST_database)-aineistoa,joka sisältää käsinkirjoitettuja numeroita 0-9 kuvina. Lataa aineisto ja jaa se opetus- ja testijoukkoihin `X_train, X_test, y_train ja  y_test` siten, 
että testijoukon osuus on 20% ja opetusjoukon osuus on 80% havainnoista. Tällä kertaa käytä suoraan kuvia `dataset.images` äläkä yksiulotteisia havaintovektoreita `dataset.data`, sillä prosessointi tehdään myöhemmin.

In [31]:

X = None
y = None

X_train, X_test, y_train, y_test = None,None,None,None
assert X and X_train

Luo aluksi pelkkä [Sequential](https://keras.io/getting-started/sequential-model-guide/)-malli, johon 
myöhemmin lisätään tasoja `.add()`-metodilla.

In [32]:
model = None
assert model

Jotta mallin syötetaso ottaisi vastaan suoraan kaksiulotteisia kuvia, lisää malliin syötetaso, joka litistää havainnot yksiuloitteisiksi. Käytä tähän [Flatten](https://keras.io/layers/core/#flatten)-tasoa ja aseta sen `input_shape`-parametriksi pikseleiden koko. `input_shape`-parametria käytetään malleissa vain ensimmäisessä tasossa, jonka jälkeen malli osaa päätellä sen myöhemmissä tasoissa. Vinkki: saat yksittäisen kuvan koon selville `.shape`-metodilla.  

In [33]:
# Lisää model-olioon taso
model.add(None)

Seuraavaksi lisää [Dense](https://keras.io/layers/core/#Dense)-taso, joka on yksinkertainen halutun kokoinen neuraalotaso. Aseta tason kooksi 100 ja lisää sille aktivointifunktioksi `"relu"` `activation`-parametrin avulla. Katso lisätietoja [dokumentaatiosta](https://keras.io/activations/) 

In [34]:
# Lisää model-olioon taso
model.add(None)

Ylisovituksen estämiseksi malleille voi olla hyvä lisätä [Dropout](https://keras.io/layers/core/#dropout)-taso. Tämä taso muuttaa satunnaisesti tietyn prosenttiosuuden painotuksista nollaksi opetusiteraatioiden aikana. Aseta pudotuksen asteeksi 0.2.

In [35]:
# Lisää model-olioon taso
model.add(None)

Mallille kerrottiin aluksi havaintojen koko `input_shape`-parametrilla ensimmäisessä tasossa. Samalla tavalla myös viimeisen tason haluttu koko on kerrottava mallille. Luokitteluongelmissa koko on tyypillisesti luokkien määrä. Painotukset halutaan kuitenkin muuttaa järkevästi tulkittaviksi, esimerkiksi todennäköisyysarvoiksi. Luokittelussa halutaan, että suurimman todennäköisyyden saanut luokka on luokittelutulos ja että kaikkien luokkien todennäköisyyslukemien summa olisi 1 jokaisella havainnolla. Tähän soveltuu mainiosti matemaattisesti [softmax](https://en.wikipedia.org/wiki/Softmax_function)-funktio.

Lisää siis viimeiseksi tasoksi [Dense](https://keras.io/layers/core/#Dense)-taso, sen kooksi haluttu luokkien lukumäärä ja sen aktivointifunktioksi `"softmax"`.

In [36]:
# Lisää model-olioon taso
model.add(None)

Malli on nyt valmis! Ennen opettamista malli täytyy vielä [koota](https://keras.io/models/sequential/#compile) (engl. compile) sopivan optimointifunktion ja häviöfunktion avulla. Käytetään tässä optimointifunktiona suosittua [`adam`](https://keras.io/optimizers/#adam)-funktiota.

Luokitteluongelman häviöfunktioksi sopii tyypillisesti joko [categorical_crossentropy](https://keras.io/losses/#categorical_crossentropy) tai [sparse_categorical_crossentropy](https://keras.io/losses/#sparse_categorical_crossentropy) riippuen siitä, missä muodossa data luokat ovat. Tähän asti luokat ovat olleet vain yksittäisiä numeroita. One-hot -koodaustapaa käytettäessä luokka ilmoitetaan eri tavalla indeksin avulla. Esimerkiksi aiemmin käytetyllä tavalla luokkien joukko [0, 2, 1] esitettäisiin one-hot -koodaustavalla:`[1 0 0],  [0 0 1], [0 1 0]`. One-hot pyrkii siis hävittämään  One-hot -koodaustavan tapauksessa `"categorical_crossentropy"` olisi soveltuva häviöfunktio, `"sparse_categorical_crossentropy"` taas sopii perinteiseen tapaan. 

Häviön lisäksi malli myös laskee metriikoita opetuksen edetessä. Tyypillisesti `"accuracy"`-metriikka riittää. Se kuvaa yleistä tarkkuutta OA.


In [85]:
# Kokoa model-olio

print(model.summary())

### Opettaminen
Mallin voi nyt opettaa. Tämä tehdään jo scikit-learn:ista tutulla `.fit()`-metodilla. Ylimääräiseksi parametriksi annetaan kuitenkin `epochs`, joka määrittää opetuksen keston koko opetusaineiston iteraatioina. Eli jos `epochs=10`, malli pääsee näkemään jokaisen opetusaineiston havainnon 10 kertaa opetuksen aikana. Liian suuri arvo voi johtaa ylisovitukseen ja liian pieni taas alisovitukseen. Valinnaisena parametrina voi antaa myös parametrin `batch_size`, eli erän koko. Erän koko kuva sitä kuinka kuinka monta havaintoa käsitellään kerrallaan. Siinä taas liian pieni arvo johtaa hitaaseen oppimiseen ja liian suuri taas voi johtaa alisovitukseen tai suorituskykyongelmiin.

Mallin opettaminen tulostaa ruudulle jatkuvasti häviön ja tarkkuuden arvoa. Nämä kuvaavat siis kyseisen epookin sisällä opetusjoukon sisäistä häviötä ja tarkkuutta. Mitä pienempi häviö ja mitä suurempi tarkkuus, niin sitä parempi malli pitäisi olla. Kuitenkin näitä arvoja ei pidä sekoittaa testausaineistolla tehtävään evaluointiin ja vasta opetuksen jälkeinen evaluointi kertoo mallin kyvystä yleistyä ongelmaan.Jos näyttää siltä, ettei häviö pienene epookkien välillä, epookkeja saattaa olla liikaa.


Opeta malli opetusjoukolla käyttäen 5 epookkia ja erän kokona 50 havaintoa.

In [89]:
# Opeta model-olio
history = model.fit(None)

Opettaminen palauttaa `history`-olion, jota voi analysoida.

In [103]:
import pandas as pd
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist = hist.set_index('epoch')
hist

In [104]:
import matplotlib.pyplot as plt

plt.plot(hist["accuracy"])
plt.title('Mallin tarkkuus')
plt.ylabel('OA')
plt.xlabel('Epookki')
plt.show()

In [105]:
plt.plot(hist["loss"])
plt.title('Mallin häviö')
plt.ylabel('Häviö')
plt.xlabel('Epookki')
plt.show()

### Testaaminen
Mallin testaamiseksi voit käyttää `evaluate()`-metodia samaan tyyliin, kuin scikit-learn -luokittelijoissa `clf.score()`-funktiota.


In [44]:
# Evaluoi malli
evaluation = None
print("Testijoukon häviö: {:.4f} OA: {:.4f}".format(evaluation[0], evaluation[1]))

Voit myös käyttää scikit-learn -kirjaston evaluointitekniikoita. Muodosta aluksi `y_pred` aineisto mallin avulla käyttäen `.predict()`-metodia.

In [61]:
y_pred = None
print("Ensimmäinen alkio:", y_pred[0])

Tarkastelemalla `y_pred`-oliota huomaat, että yksittäinen alkio onkin luokan sijasta lista todennäköisyyksiä<sup>1</sup>. 
Ensimmäinen luku on luokan 0 todennäköisyys, toinen luku luokan 1 todennäköisyys ja niin edelleen. Luokitellun luokan saamiseksi on siis otettava suurimman todennäköisyyden saaneen indeksi. 
Tähän voi käyttää [`np.argmax()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html)-metodia.


<sup>1</sup>Myös Scikit-learn-kirjastosta on mahdollista saada todennäköisyysarvot luokkien lisäksi. Joidenkin
 algoritmien, kuten SVM:n tapauksessa tällöin on annettava parametri `probability=True` mallia luotaessa.

In [62]:
import numpy as np
y_pred_probs = y_pred
y_pred = np.argmax(y_pred_probs, axis=1)
print("Suurimman todennäköisyyden saanut luokka: {}, sen saama todennäköisyys: {:.4f}".format(
    y_pred[0], y_pred_probs[0][y_pred[0]]))

Muodosta luokitteluraportti ja sekaannusmatriisi.

In [1]:
from sklearn.metrics import classification_report, confusion_matrix
from utils import plot_confusion_matrix

report = None
cm = None
plot_confusion_matrix(cm, list(range(10)), list(range(10)), normalize=True)

assert report
print(report)

## Mallin muokkaaminen
Kokeile seuraavaksi muokata mallia paremmaksi. Kopioi malli osa osalta alle `build_fn()`-funktion sisälle, jotta voit muuttaa parametreja helpommin. Käytä funktion parametreja ja voit lisätä omia parametreja muokkaamisen helpottamiseksi, mutta muist antaa niille oletusarvo. Voit lisätä malliin uusia (Dense-) tasoja ja voit muuttaa niiden kokoa ja poiston määrää. Muuta myös erien kokoa ja epookkien määriä. Älä kuitenkaan käytä liian isoja epookkilukuja (>20) tai neuronitasojen kokoja (>10000), jottei mallin opettaminen hidastu liikaa.

In [84]:
def build_fn(param1=None, param2=None):
    # Kopioi malli yllä olevista soluista sisältäen koontivaiheen (model.compile())
    model = None
    
    print(model.summary())
    return model
    
# Muokkaa parametrejä
model = build_fn(param1=None, param2=None)
model.fit(X_train, y_train, epochs=5, batch_size=50)
evaluation = model.evaluate(X_test, y_test, verbose=False)
print("Testijoukon häviö: {:.4f} OA: {:.4f}".format(evaluation[0], evaluation[1]))

### Mallin tallentaminen
Kuten Scikit-learn:in tapauksessa, myös Keras-malleja voi tallentaa levylle. 
Neuroverkkojen opettaminen on usein hidasta puuhaa, mutta onneksi myös välituloksia opetuksessa 
voi tallentaa ja opetusta voi tarvittaessa jatkaa edellisestä kohdasta tallentamalla mallin painot.  

[Dokumentaation](https://keras.io/callbacks/#modelcheckpoint) perusteella luo `keras.callbacks.ModelCheckpoint`-funktio, joka tallentaa vain parhaat painot tiedostoon "keras-mnist-model-best-weights.ckpt"

In [86]:
checkpoint_location = "keras-mnist-model-best-weights.ckpt"

cp_callback = None
assert cp_callback

model = build_fn()
model.fit(X_train, y_train, epochs=5, batch_size=50, callbacks = [cp_callback])
evaluation = model.evaluate(X_test, y_test, verbose=False)
print("Testijoukon häviö: {:.4f} OA: {:.4f}".format(evaluation[0], evaluation[1]))

In [88]:
# Evaluoi tyhjää mallia
model = build_fn()
evaluation = model.evaluate(X_test, y_test, verbose=False)
print("Painottoman mallin Testijoukon häviö: {:.4f} OA: {:.4f}".format(evaluation[0], evaluation[1]))

# Evaluoi mallia painojen lataamisen jälkeen
model.load_weights(checkpoint_location)
evaluation = model.evaluate(X_test, y_test, verbose=False)
print("Painnollisen mallin Testijoukon häviö: {:.4f} OA: {:.4f}".format(evaluation[0], evaluation[1]))

Lue halutessasi lisää mallin tallentamisesta [täältä](https://www.tensorflow.org/tutorials/keras/save_and_restore_models).

## Scikit-learn yhteensopivuus
Äskeinen hyperparametrien optimointi saattoi tuntua aika manuaaliselta edellisten harjoitusten 
automatisoitujen keinojen jälkeen. Pystyisiköhän samoja menetelmiä käyttämään myös Keras-mallien kanssa? 
Kyllä [pystyy](https://keras.io/scikit-learn-api/)! Samoin Keras-mallin voi ottaa käyttöön
 laajemmassa Scikit-learn ympäristössä  ja siihen voi jopa soveltaa 
 [putkitusta](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), 
 mikä tuo paljon hyötyjä esimerkiksi datan esikäsittelyn (esim. standardtointi, pca) osalta.

Luo ensin `clf`-olio [dokumentaation](https://keras.io/scikit-learn-api/) 
avulla antamatta mitään muita parametreja, kuin `build_fn`.



In [2]:
clf = None
assert clf

Luo sitten parametriverkko ja [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
. Luomasi `build_fn` parametrien lisäksi voit antaa parametreina myös opettamiseen tarvittavia parametreja, kuten `"epochs"` ja `"batch_size"`. Muista, että GridSearchCV käyttää ristivalidointia (CV=3 oletuksena), eli liian paljon eri aikaavieviä parametrikokeiluja ei kannata tehdä kerralla.

In [82]:
from sklearn.model_selection import GridSearchCV
parameters = None
clf = GridSearchCV(clf, parameters)
clf.fit(X_train, y_train)
print("Parhaat parametrit: ", clf.best_params_)
print("Paras opetus OA: {:.4f}".format(clf.best_score_))
print("OA: {:.4f}".format(clf.score(X_test, y_test)))


## Lisenssi
Osa tämän harjoituksen esimerkeistä on muokattu sivun https://www.tensorflow.org/tutorials esimerkeistä.


In [25]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.